In [ ]:
from collections import defaultdict

import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt

# from importlib import reload
# import feature_extraction
# reload(feature_extraction)
from plotting import (
    plot_series,
    plot_eeg_topomap_one_block,
    plot_eeg_topomap_all_blocks,
    plot_k_chaneels_by_r_value,
    plot_eeg_pearson_correlation_table,
    get_eeg_pearson_correlation_series_all_blocks,
    plot_time_series_by_epoch,
    plot_scatter_by_marker,
)
from biomarkers import (
    EEG,
    EMG,
    BP,
    EOG,
    ECG,
    TREV,
    GSR,
    EGG,
)
from feature_extraction import (
    Feature,
    EEG_BANDS,
    FEATURE_TO_FUNC,
)
from data_utils import (
    load_data_from_dir,
    get_all_behaviors_labels,
    get_all_features_by_marker,
    extract_labels,
    extract_features_by_channel,
    concatenate_all_data,
)
from calculate_correlation import (
    STAT_FEATURES,
    get_all_behaviors_feature_to_pc_by_markers,
    get_all_trials_average_rp_values,
)

from labels import get_categorical_labels, print_label_count


In [ ]:
ALL_DIRS = ["../2000_CleanData", "../2001_CleanData", "../1004_CleanData"]

dir_to_data = {}
for dir_name in ALL_DIRS:
    all_data = load_data_from_dir(dir_name)
    dir_to_data[dir_name] = all_data

In [ ]:
from data_utils import (
    extract_features,
    extract_labels,
)

features = STAT_FEATURES

""" extract features from physiological signals
"""
feature_names = []
all_feature_array = []
for m in [EOG.__name__, EMG.__name__]:
    f_array, names = extract_features(m, dir_to_data, features)
    all_feature_array.append(f_array)
    feature_names.extend(names)

all_feature_array = np.concatenate(all_feature_array, axis=-1)
all_label_array = extract_labels(dir_to_data)

print(all_feature_array.shape, len(feature_names), all_label_array["valence"].shape)


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

bahavior = "arousal"
X, y = all_feature_array, all_label_array[bahavior]
print(X.shape, y.shape)
# (TODO) Using sklearn StandardScaler to normlize the data
# train_test_split
X_train, X_test = X[:312], X[312:]
y_train, y_test = y[:312], y[312:]
# lightGBM
# Plot confusion metrics
est = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss="ls"
).fit(X_train, y_train)
print(mean_squared_error(y_test, est.predict(X_test)))

feat_importances = pd.Series(est.feature_importances_, index=feature_names)
feat_importances.nlargest(10).plot(kind="barh", title=f"{bahavior} Feature Importance")


In [ ]:
""" concatenate raw signal (x)
"""
data_list = []
for m in [EEG.__name__, EGG.__name__]:  # EOG.__name__, , EMG.__name__, EGG.__name__
    all_participants_data, condition_to_labels = concatenate_all_data(dir_to_data, m)
    all_epoch_data = np.swapaxes(
        all_participants_data, 0, -1
    )  # (num_channels, num_data_points, num_epochs) => (num_epochs, num_data_points, num_channels)

    data_list.append(all_epoch_data)

data_array = np.concatenate(data_list, axis=2)

In [ ]:
from tensorflow.keras import utils as np_utils
from models import cnn_model, train_with_cnn

""" prepare labels (y)
"""
# to one-hot encoding vector
label_list = get_categorical_labels(condition_to_labels, valence_threshold=0.65)
groups_list = [[i] for i, j in enumerate(label_list)]
group_array = np.hstack(groups_list)
label_array = np_utils.to_categorical(
    label_list, num_classes=4
)  # nvla, nvha, hvla, hvha
# label_array = np.array(label_list['arousal'])

print_label_count(label_list)
print(data_array.shape, label_array.shape, group_array.shape)
num_channel = data_array.shape[2]

model = cnn_model(12288, num_channel)
model.summary()

In [ ]:
accuracy = train_with_cnn(12288, num_channel, data_array, label_array, group_array)

In [ ]:
print(accuracy)
print(np.mean(accuracy))

In [ ]:
features = STAT_FEATURES
marker = ECG.__name__

""" extract features from physiological signals
"""
dir_name_to_features = extract_features_by_channel(marker, dir_to_data, features, 0)
dir_name_to_labels = extract_labels(dir_to_data)

In [ ]:
""" extract correlation from physiological signals
"""
num_channels = 4
num_blocks = 0
dir_name_to_ctf = {}
for dir_name, all_data in dir_to_data.items():
    dir_name_to_ctf[dir_name] = get_all_behaviors_feature_to_pc_by_markers(
        all_data, marker, features, num_channels, num_blocks
    )

avg_condition_to_features = get_all_trials_average_rp_values(
    dir_name_to_ctf, features, "pearson"
)
spearman_corr = get_all_trials_average_rp_values(dir_name_to_ctf, features, "spearman")
for b, feature_to_pc in avg_condition_to_features.items():
    for f, pc in feature_to_pc.items():
        avg_condition_to_features[b][f] = np.hstack((pc, spearman_corr[b][f]))

In [ ]:
all_data = dir_to_data["../2000_CleanData"]
channel_names = all_data["audio_hvla"].get_chanlocs(marker)

""" plot the time series given the marker and block
"""
# plot_time_series_by_epoch(all_data['audio_hvla'], 'EMG', 'audio_hvla', 0)


""" plot the single channel correlation table
"""
# channel = 3
# for condition, feature_to_pc in avg_condition_to_features.items():
#     label = f"{channel_names[channel]} {condition}"
#     plot_correlation_table_by_channel(
#         label, feature_to_pc, ["pearson r", "pearson p", "spearman r", "spearman p"], features, channel, True
#     )

""" plot the top channel correlation table given r values
"""
# for condition, feature_to_pc in avg_condition_to_features.items():
#     plot_k_chaneels_by_r_value(feature_to_pc, channel_names, features, condition, True, 2)
#     plot_k_chaneels_by_r_value(feature_to_pc, channel_names, features, condition, False, 2)


""" plot the top channel correlation table with different blocks
"""
# all_block_names = list(all_data.keys())
# all_block_names.sort()
# for condition, feature_to_pc in avg_condition_to_features.items():
#     plot_eeg_pearson_correlation_table(condition, feature_to_pc, all_block_names, 1)

""" plot the series for the top k channels
"""
# define number of rows and columns for subplots
# nrow = 3
# ncol = 2
# for condition, feature_to_pc in avg_condition_to_features.items():
#     print(f"{condition}")
#     ser_list = get_eeg_pearson_correlation_series_all_blocks(feature_to_pc, channel_names, k=20)
#     plot_series(nrow, ncol, ser_list)

""" plot the topography for eeg
"""
# for condition, feature_to_pc in avg_condition_to_features.items():
#     plot_eeg_topomap_all_blocks(condition, feature_to_pc)
#     for f in list(EEG_BANDS.keys()):
#         plot_eeg_topomap_one_block(condition, f, feature_to_pc, all_block_names)

In [ ]:
""" extract features from physiological signals
"""
dir_name_to_features = {}
dir_name_to_labels = {}
channel_num = 1
for dir_name, all_data in dir_to_data.items():
    feature_to_value = get_all_features_by_marker(
        all_data, marker, features, channel_num
    )

    dir_name_to_features[dir_name] = feature_to_value
    dir_name_to_labels[dir_name] = get_all_behaviors_labels(all_data)

features_to_trials = defaultdict()
for dir_name, fv in dir_name_to_features.items():
    for f, v in fv.items():
        if f not in features_to_trials:
            features_to_trials[f] = defaultdict()
        if dir_name not in features_to_trials[f]:
            features_to_trials[f][dir_name] = defaultdict()

        features_to_trials[f][dir_name] = v


In [ ]:
""" plot the scattor for physiological signals
"""
# plot_scatter_by_marker(marker, features_to_trials, dir_name_to_labels, channel_names[channel_num])

In [ ]:
def get_stats_name(i) -> str:
    pos = i % 11
    return STAT_FEATURES[pos]


def get_feature_names(features):
    index = []
    for i, j in enumerate(features):
        stat_name = get_stats_name(i)
        if i < 11:
            name = "BP"
        elif i < 22:
            name = "ECG"
        elif i < 1430:
            channel = int((i - 22) / 11) + 1
            name = f"EEG {channel}"
        elif i < 1441:
            name = "EGG"
        elif i < 1463:
            channel = int((i - 1441) / 11) + 1
            name = f"EMG {channel}"
        elif i < 1485:
            channel = int((i - 1463) / 11) + 1
            name = f"EOG {channel}"
        elif i < 1496:
            name = "GSR"
        elif i < 1507:
            name = "Resp"
        elif i < 1518:
            name = "TREV"
        else:
            channel = int((i - 1518) / 6) + 1
            name = f"EEG {channel}"
            stat_name = EEG_BANDS[(i - 1518) % 6]

        index.append(f"{name} {stat_name}")
    return index

In [ ]:
""" with logistic regression
"""
# from models import train_with_logistic

# label_name = "attention"
# best_model = train_with_logistic(all_features, label_name, condition_to_labels, group_array)
# name_to_transformed = get_tranformed_labels(condition_to_labels)

# # assume bigger coefficents has more contribution to the model
# # but have to be sure that the features has THE SAME SCALE otherwise this assumption is not correct.
# importance = best_model["classifier"].coef_[0]

# feat_importances = pd.Series(importance, index=get_feature_names(importance))
# feat_importances.nlargest(10).plot(
#     kind="barh", title=f"{label_name} Feature Importance"
# )